In [1]:
import src.view.pnl_view as pnlview
from multiprocessing import Pool, Process, Manager, cpu_count
from src.core.logger import Logger
from src.core.stopwatch import Stopwatch
from src.core.stopwatch_logger import StopwatchLogger
from src.synthetic_book_merger import SyntheticBookMerger
from src.synthetic_security_loader import SyntheticSecurityLoader
from src.sniper_opp_loader import SniperOppLoader
# import dask
from dask import dataframe as dask_df
# from pandarallel import pandarallel

from src.view import data_view as dview
from src.view.opp_view import OppView
from src.view.cell_view import CellView
from src.view.color import Color, Colors, WebColors
from src.pcap_location_params import PCapLocationParams
from src.core.postgres_connection import PostgresConnection
from src.view.cell_view import CellView
from src.market_date import *

import pandas as pd
import numpy as np
import sqlite3
import time
import matplotlib.pyplot as plt
import os.path

# pandarallel.initialize(progress_bar=True, use_memory_fs=True)
pd.options.display.width = 920
pd.options.display.max_columns = 500
pd.options.display.max_rows = 220
pd.options.display.max_rows = 263

In [479]:
import src.view.opp_view
import src.view.data_view as dview
import src.core.sort_utils as sort_utils
import src.view.edge_summary_view
import src.synthetic_security_loader
importlib.reload(module=src.view.opp_view)
importlib.reload(module=dview)
importlib.reload(module=src.view.edge_summary_view)
importlib.reload(module=src.synthetic_security_loader)
from src.view.opp_view import OppView
from src.view.edge_summary_view import get_edge_summary
from src.synthetic_security_loader import SyntheticSecurityLoader

In [424]:
# pd.read_feather(PCapLocationParams(340, date='2021-07-19').feather_file_path('security')).sort_values(['symbol'], ascending=False).head(500).style

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
2220,11663,ZWZ3-ZWN4,2023-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
1265,53331,ZWZ3-ZWK4,2023-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
1339,11888,ZWZ3-ZWH4,2023-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
1121,11548,ZWZ3,2023-12-14 18:01:00,FUTURE,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.250000,0,70
3580,71046,ZWZ2-ZWZ3,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
750,86518,ZWZ2-ZWU3,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
1944,78739,ZWZ2-ZWN4,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
3353,26705,ZWZ2-ZWN3,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
3541,88719,ZWZ2-ZWK4,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70
742,58745,ZWZ2-ZWK3,2022-12-14 18:01:00,SPREAD,FUT,ZW,ZW,True,K,1.000000,50.000000,0.250000,0.000000,2,70


In [338]:
synth_secs = SyntheticSecurityLoader(340).result().set_index('id')
synth_secs['symbol'] = pd.Categorical(synth_secs['symbol'], categories=opp_bk_edge_summ.index.get_level_values('symbol').categories)
# synth_secs = synth_secs.set_index('symbol')

In [473]:
opp_bk = opp_w_bk_root.copy()
opp_bk['t_tod'] = opp_bk['t_time'] - opp_bk.index.get_level_values('market_date') - pd.Timedelta('1d')
ov = src.view.opp_view.to_opp_view(opp_bk)
src.view.opp_view.display_opp_view(ov)

In [480]:
opp_bk= opp_w_bk_root.copy()

opp_bk = pd.merge(
    opp_bk.reset_index('symbol').rename(columns={'symbol': 'sym'}),
    synth_secs, left_on='sym', right_on='symbol', how='left')\
    .drop(columns=['symbol', 'sym'])\
    .set_index(opp_bk.index)

opp_bk['nmreq_edge'] = opp_bk['non_member_fee']
opp_bk['mreq_edge'] = opp_bk['member_fee']
opp_bk['req_edge'] = opp_bk['nmreq_edge']
opp_bk_edge_summ = get_edge_summary(opp_bk, cpp=50.0)
opp_bk_edge_summ

-0.50  -0.25  0.00  0.25  0.50  1.00   net  net_cash
market_date symbol                                                                                                    
2021-05-02  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 0    NaN    2.0   NaN   1.0   NaN   NaN -0.25     -12.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z... 0    1.0    NaN   1.0   NaN   NaN   NaN -1.00     -50.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)             0    1.0    2.0   NaN   2.0   1.0   NaN  0.00       0.0
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)             0    2.0    1.0   NaN   NaN   NaN   NaN -1.50     -75.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1              0    1.0    NaN   1.0   2.0   NaN   NaN  0.00       0.0
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1              0    2.0    1.0   1.0   1.0   NaN   NaN -1.00     -50.0
            +ZSN1 -(ZSN1-ZSX1) -ZSX1                           0    1.0    2.0   1.0   NaN   NaN   NaN -1.00     -50.0
            +ZSQ1 -(ZSQ1-ZSU1) -ZSU1                           0    1.0    NaN   1.0   NaN   NaN   NaN -0.50     -25.0
2021-05-03  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 0    NaN    NaN   NaN   NaN   1.0   NaN  0.50      25.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)             0    NaN    NaN   2.0   1.0   NaN   NaN  0.25      12.5
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)             0    NaN    NaN   1.0   NaN   NaN   NaN  0.00       0.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1              0    NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1              0    2.0    NaN   1.0   NaN   NaN   NaN -1.00     -50.0
            +ZSN1 -(ZSN1-ZSX1) -ZSX1                           0    NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN2 -(ZSN2-ZSX2) -ZSX2                           0    NaN    NaN   NaN   2.0   NaN   NaN  0.50      25.0
2021-05-04  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 0    NaN    NaN   1.0   1.0   1.0   NaN  0.75      37.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z... 0    NaN    NaN   NaN   1.0   NaN   NaN  0.25      12.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)             0    NaN    1.0   4.0   3.0   1.0   NaN  1.00      50.0
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)             0    NaN    1.0   1.0   2.0   NaN   NaN  0.25      12.5
            +(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-Z... 0    1.0    NaN   NaN   NaN   1.0   NaN -0.50     -25.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1              0    NaN    1.0   1.0   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSQ1) -ZSQ1                           0    NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1              0    NaN    NaN   1.0   NaN   1.0   NaN  0.50      25.0
            +ZSN2 -(ZSN2-ZSX2) -ZSX2                           0    1.0    1.0   3.0   3.0   NaN   NaN -0.25     -12.5
            +ZSQ1 -(ZSQ1-ZSU1) -ZSU1                           0    NaN    NaN   1.0   NaN   NaN   NaN  0.00       0.0
2021-05-05  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 0    1.0    2.0   NaN   2.0   1.0   NaN  0.00       0.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z... 0    1.0    NaN   NaN   1.0   NaN   1.0  0.50      25.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)             0    1.0    2.0   7.0   2.0   NaN   1.0  0.25      12.5
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)             0    2.0    2.0   3.0   3.0   NaN   NaN -1.00     -50.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1              0    NaN    1.0   1.0   5.0   1.0   1.0  3.00     150.0
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1              0    1.0    1.0   2.0   3.0   1.0   1.0  2.00     100.0
            +ZSN1 -(ZSN1-ZSX1) -ZSX1                           0    NaN    NaN   NaN   2.0   NaN   NaN  0.50      25.0
            +ZSN2 -(

In [486]:
es[['net_cash', 'month', 'sym']].groupby(['sym', 'month']).sum()

,,net_cash
sym,month,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),5,775.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),5,625.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),5,1187.5
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),5,-100.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),5,-437.5
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,5,25.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,5,-50.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,5,62.5
+ZSN1 -(ZSN1-ZSX1) -ZSX1,5,-112.5


In [485]:
es['month'] = pd.DatetimeIndex(es.index.get_level_values('market_date')).month
es_syms = es.index.get_level_values('symbol')
es['sym'] = es_syms

def get_monthly_pnl(x):
    return pd.DataFrame(data={str(m): [x.loc[x.name,m]['net_cash'].sum() if (x.name, m) in x.index else 0.0] for m in range(4,8)})
    
es[['net_cash', 'month', 'sym']].groupby(['sym', 'month']).sum().groupby(level='sym').apply(get_monthly_pnl)
# es.reset_index('symbol')
# es.groupby(level='symbol').sum()
# len(es.index.unique())

,,4,5,6,7
sym,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),0,0.0,775.0,0.0,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),0,0.0,625.0,0.0,0.0
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),0,0.0,1187.5,0.0,0.0
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),0,0.0,-100.0,0.0,0.0
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),0,0.0,-437.5,0.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,0,0.0,25.0,0.0,0.0
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,0,0.0,-50.0,0.0,0.0
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,0,0.0,62.5,0.0,0.0
+ZSN1 -(ZSN1-ZSX1) -ZSX1,0,0.0,-112.5,0.0,0.0


In [483]:
es = opp_bk_edge_summ.head(100).copy()
sdict = dict([(c, dview.fmt_float_lambda()) for c in ['net', 'net_cash']] + [(f'{e * .25:.2f}', dview.fmt_float_lambda(ndec=0)) for e in range(-2, 5)])
es.style.format(sdict)

In [378]:
opp_dur_buckets = {
    pd.Timedelta('0us'): CellView(),
    pd.Timedelta('50us'): CellView(bg=Colors.gold()),
    pd.Timedelta('100us'): CellView(bg=WebColors.sky_blue()),
    pd.Timedelta('500us'): CellView(bg=Colors.lightgreen_pastel())
}

edge_buckets = {
    -0.25: CellView(fw = 'bold', bg = Colors.darkred()),
    0.0: CellView(),
    0.25: CellView(fg=Colors.blue(), fw='bold'),
    0.50: CellView(fg=Colors.blue(), fw='bold', bg=WebColors.sky_blue()),
    1.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.gold()),
    2.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.lightgreen_pastel()),
}

opp_bk= opp_w_bk_root.copy()
# opp_bk = opp_bk.loc[:,['+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)', '+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1'], :]
opp_bk['lsn_delta'] = opp_bk['opp_dur_thru'] - opp_bk['opp_dur_thru_lsn']
opp_bk = opp_bk.loc[(opp_bk['opp_dur_lsn'] <= pd.Timedelta('500us'))]
opp_bk = opp_bk.loc[opp_bk['lsn_delta'] < pd.Timedelta('1ms')]
opp_bk['t_tod'] = opp_bk['t_time'] - opp_bk.index.get_level_values('market_date') - pd.Timedelta('1d')

def get_pnl(x):
    obtainable = x.loc[x['opp_dur_thru_lsn'] >= pd.Timedelta('55us')]
    if len(obtainable) == 0:
        return x.head(1)
    idx = obtainable[['edge']].idxmax()
    rv = obtainable.loc[idx]
    return rv

opp_bk = opp_bk.groupby(level=[0,1,2]).apply(get_pnl)
opp_bk = opp_bk.set_index([opp_bk.index.get_level_values(i) for i in range(3, 7)])
opp_bk = dview.with_groupby_origin_col(opp_bk, level=['market_date', 'symbol'])

microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'lsn_delta', 'opp_dur_thru', 'opp_dur_thru_lsn']
opp_bk = opp_bk[['is_origin', 't_time', 'bk_dur', 'lsn_delta', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1']]

ov = OppView(opp_bk.head(320), opp_durs=opp_dur_buckets, edges=edge_buckets, opp_origin_css='opp-origin')
microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'lsn_delta', 'opp_dur_thru', 'opp_dur_thru_lsn']
ov.display(verbose=None,
           columns =['is_origin', 't_tod', 'bk_dur', 'lsn_delta', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1'])\
    .format({c: lambda x: dview2.TimedeltaView(x, fmt='micros') for c in microsecond_cols}, subset=microsecond_cols)\
    .hide_columns(['is_origin', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru'])

# opp_bk['edge'].sum() * 4 * 12.5
# opp_bk.head(100).groupby(level=[0,1,2]).apply(lambda x: x.head(3))
# ov.opp_view

In [376]:
opp_dur_buckets = {
    pd.Timedelta('0us'): CellView(),
    pd.Timedelta('50us'): CellView(bg=Colors.gold()),
    pd.Timedelta('100us'): CellView(bg=WebColors.sky_blue()),
    pd.Timedelta('500us'): CellView(bg=Colors.lightgreen_pastel())
}

edge_buckets = {
    -0.25: CellView(fw = 'bold', bg = Colors.darkred()),
    0.0: CellView(),
    0.25: CellView(fg=Colors.blue(), fw='bold'),
    0.50: CellView(fg=Colors.blue(), fw='bold', bg=WebColors.sky_blue()),
    1.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.gold()),
    2.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.lightgreen_pastel()),
}

opp_bk= opp_w_bk_root.copy()
# opp_bk['lst_delta_us'] = opp_bk['lst_delta'].apply(lambda x: x.microseconds)
# opp_bk = opp_bk.loc[:,['+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)', '+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1'], :]
# opp_w_bk['opp_start_eid'] = opp_w_bk.index.get_level_values('opp_start_eid')
# opp_bk = opp_bk.loc[opp_bk['opp_dur_lsn'] > pd.Timedelta(0, unit='us')]
# ov = opp_bk.loc[opp_bk['opp_dur_thru_lsn'] >= pd.Timedelta('0ns')]\
opp_bk['lsn_delta'] = opp_bk['opp_dur_thru'] - opp_bk['opp_dur_thru_lsn']
opp_bk = opp_bk.loc[(opp_bk['opp_dur_lsn'] <= pd.Timedelta('500us'))]\
#     .groupby(level=[0,1,2]).apply(lambda x: x.head(10))
opp_bk = opp_bk.loc[opp_bk['lsn_delta'] < pd.Timedelta('1ms')]
# ov = ov.set_index([ov.index.get_level_values(i) for i in range(3, 7)])
def get_pnl(x):
    obtainable = x.loc[x['opp_dur_thru_lsn'] >= pd.Timedelta('55us')]
    # obtainable['pnl'] = obtainable['edge'].max()
    if len(obtainable) == 0:
        return x.head(1)
    idx = obtainable[['edge']].idxmax()
    rv = obtainable.loc[idx]
#     print(f'Good: {rv.index}')
    return rv

opp_bk = opp_bk.groupby(level=[0,1,2]).apply(get_pnl)
opp_bk = opp_bk.set_index([opp_bk.index.get_level_values(i) for i in range(3, 7)])

# ov = ov.groupby(level='market_date').apply(add_is_origin_col)
opp_bk = dview.with_groupby_origin_col(opp_bk, level=['market_date', 'symbol'])
# opp_bk['is_origin'] = False
# opp_bk = opp_bk.groupby(level='market_date').apply(add_is_origin_col)
microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'lsn_delta', 'opp_dur_thru', 'opp_dur_thru_lsn']
opp_bk = opp_bk[['is_origin', 't_time', 'bk_dur', 'lsn_delta', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1']]

# sdict = dview.get_style_dict(ov)
# sdict.update({c: lambda x: dview2.TimedeltaView(x, fmt='micros') for c in microsecond_cols})
# sdict.update({'edge': '{:.2f}'})
# ov.style.format(sdict).hide_columns(['opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru'])

ov = OppView(opp_bk.head(320), opp_durs=opp_dur_buckets, edges=edge_buckets, opp_origin_css='opp-origin')
microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'lsn_delta', 'opp_dur_thru', 'opp_dur_thru_lsn']
ov.display(verbose=None,
           columns =['is_origin', 't_tod', 'bk_dur', 'lsn_delta', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1'])\
    .format({c: lambda x: dview2.TimedeltaView(x, fmt='micros') for c in microsecond_cols}, subset=microsecond_cols)\
    .hide_columns(['is_origin', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru'])

# opp_bk['edge'].sum() * 4 * 12.5
# opp_bk.head(100).groupby(level=[0,1,2]).apply(lambda x: x.head(3))
# ov.opp_view

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 51386 entries, (Timestamp('2021-05-02 00:00:00'), '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)', 11561, 11561) to (Timestamp('2021-05-31 00:00:00'), '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1', 12075838, 12075932)
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   t_time            51386 non-null  datetime64[ns] 
 1   id                51386 non-null  int32          
 2   bid_p_1           51386 non-null  float32        
 3   bid_q_1           51386 non-null  uint32         
 4   bid_p_2           51386 non-null  float32        
 5   bid_q_2           51386 non-null  uint32         
 6   bid_p_3           51367 non-null  float32        
 7   bid_q_3           51386 non-null  uint32         
 8   ask_p_1           51386 non-null  float32        
 9   ask_q_1           51386 non-null  uint32         
 10  ask_p_2           51386 non-null  float32      

-0.50  -0.25  0.00  0.25  0.50  1.00   net  net_cash
market_date symbol                                                                                                  
2021-05-02  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z...    NaN    3.0   1.0   9.0   1.0   NaN  2.00     100.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z...    1.0    NaN   2.0   2.0   NaN   NaN -0.50     -25.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)                1.0    2.0   NaN   2.0   1.0   NaN  0.00       0.0
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)                2.0    1.0   NaN   NaN   NaN   NaN -1.50     -75.0
            +(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-Z...    NaN    NaN   1.0   1.0   NaN   NaN  0.25      12.5
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1                 1.0    NaN   1.0   2.0   NaN   NaN  0.00       0.0
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1                 2.0    1.0   1.0   1.0   NaN   NaN -1.00     -50.0
            +ZSN1 -(ZSN1-ZSX1) -ZSX1                              1.0    2.0   1.0   NaN   NaN   NaN -1.00     -50.0
            +ZSQ1 -(ZSQ1-ZSU1) -ZSU1                              1.0    NaN   1.0   NaN   NaN   NaN -0.50     -25.0
2021-05-03  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z...    NaN    2.0   2.0  17.0   1.0   NaN  4.25     212.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)                NaN    NaN   2.0   1.0   NaN   NaN  0.25      12.5
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)                NaN    NaN   1.0   NaN   NaN   NaN  0.00       0.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1                 NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1                 2.0    NaN   1.0   NaN   NaN   NaN -1.00     -50.0
            +ZSN1 -(ZSN1-ZSX1) -ZSX1                              NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN2 -(ZSN2-ZSX2) -ZSX2                              NaN    NaN   NaN   2.0   NaN   NaN  0.50      25.0
2021-05-04  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z...    NaN    6.0   6.0  25.0   1.0   NaN  5.25     262.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z...    NaN    NaN   5.0   7.0   1.0   NaN  2.25     112.5
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)                NaN    1.0   4.0   3.0   1.0   NaN  1.00      50.0
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)                NaN    1.0   1.0   2.0   NaN   NaN  0.25      12.5
            +(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-Z...    1.0    NaN   1.0   8.0   1.0   NaN  1.50      75.0
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1                 NaN    1.0   1.0   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSQ1) -ZSQ1                              NaN    1.0   NaN   NaN   NaN   NaN -0.25     -12.5
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1                 NaN    NaN   1.0   NaN   1.0   NaN  0.50      25.0
            +ZSN2 -(ZSN2-ZSX2) -ZSX2                              1.0    1.0   3.0   3.0   NaN   NaN -0.25     -12.5
            +ZSQ1 -(ZSQ1-ZSU1) -ZSU1                              NaN    NaN   1.0   NaN   NaN   NaN  0.00       0.0
2021-05-05  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z...    2.0    4.0   1.0  16.0   1.0   NaN  2.50     125.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-Z...    1.0    1.0   3.0   2.0   NaN   1.0  0.50      25.0
            +(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)                1.0    2.0   7.0   2.0   NaN   1.0  0.25      12.5
            +(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2)                2.0    2.0   3.0   3.0   NaN   NaN -1.00     -50.0
            +(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-Z...    1.0    NaN   1.0   3.0   NaN   NaN  0.25      12.5
            +ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1                 NaN    1.0   1.0   5.0   1.0   1.0  3.00     150.0
            +ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1                 1.0    1.0   2.0   3

In [345]:
# opp_bk_edge_short = opp_bk_edge_summ.head(10).copy()
sdict = {f'{e:.2f}': lambda x: '' if np.isnan(x) else int(x) for e in [-.5, -.25, 0.0, .25, .5, 1.0]}
sdict.update({c: '{:.2f}' for c in ['net', 'net_cash', 'nm_net_cash', 'm_net_cash', 'non_member_fee', 'member_fee']})
opp_bk_edge_short = opp_bk_edge_summ.head(10).copy()
# opp_bk_edge_short['net_cash_nm'] = opp_bk_edge_short.groupby(level='symbol').apply(lambda x: synth_secs.loc[synth_secs['symbol'] == x]['non_member_fee'])
# opp_bk_edge_summ.style.format(sdict)
opp_bk_edge_short.groupby(level=['symbol']).apply(lambda x: synth_secs.loc[synth_secs['symbol'] == x.name][['non_member_fee']])


,,non_member_fee
symbol,id,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),2,15.36
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),3,15.36
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),1,11.52
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),9,11.52
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),5,15.36
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,4,11.58
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,7,7.74
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,11,11.58
+ZSN1 -(ZSN1-ZSX1) -ZSX1,10,7.74


In [346]:
opp_bk_edge_summ.loc[opp_bk_edge_summ['net'] < 0].sort_values('net_cash', ascending=True).style.format(sdict)

In [161]:
import math
# import src.view.data_view10 as dview2
import src.view.opp_view18 as oview2
opp_dur_buckets = {
    pd.Timedelta('0us'): CellView(),
    pd.Timedelta('50us'): CellView(bg=Colors.gold()),
    pd.Timedelta('100us'): CellView(bg=WebColors.sky_blue()),
    pd.Timedelta('500us'): CellView(bg=Colors.lightgreen_pastel())
}

edge_buckets = {
    -0.25: CellView(fw = 'bold', bg = Colors.darkred()),
    0.0: CellView(),
    0.25: CellView(fg=Colors.blue(), fw='bold'),
    0.50: CellView(fg=Colors.blue(), fw='bold', bg=WebColors.sky_blue()),
    1.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.gold()),
    2.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.lightgreen_pastel()),
}

opp_bk= opp_w_bk_root.copy()
# opp_bk['lst_delta_us'] = opp_bk['lst_delta'].apply(lambda x: x.microseconds)
opp_bk = opp_bk.loc[:,'+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',:]
# opp_w_bk['opp_start_eid'] = opp_w_bk.index.get_level_values('opp_start_eid')
# opp_bk = opp_bk.loc[opp_bk['opp_dur_lsn'] > pd.Timedelta(0, unit='us')]
# ov = opp_bk.loc[opp_bk['opp_dur_thru_lsn'] >= pd.Timedelta('0ns')]\
def add_is_origin(x):
    x['is_origin'] = False
    x.iat[0, x.columns.get_loc('is_origin')] = True
#     x['is_origin'] = x.name[2] >= x['synth_bk_eid']
    return x.head(10)
ov = opp_bk.loc[(opp_bk['opp_dur_lsn'] <= pd.Timedelta('500us')) & (opp_bk['edge'] >= 0)]\
    .groupby(level=[0,1,2]).apply(add_is_origin)
ov['lsn_delta'] = ov['opp_dur_thru'] - ov['opp_dur_thru_lsn']
ov = ov.loc[ov['lsn_delta'] < pd.Timedelta('1ms')]

ov = OppViewDbg(
    ov.set_index([ov.index.get_level_values(i) for i in range(3, 7)])\
    .head(200), opp_durs=opp_dur_buckets, edges=edge_buckets, opp_origin_css='opp-origin')
# ov = OppView(opp_w_bk.head(120))
microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'lsn_delta', 'opp_dur_thru', 'opp_dur_thru_lsn']
ov.display(verbose=None,
           columns =['is_origin', 't_tod', 'bk_dur', 'lsn_delta', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1'])\
    .format({c: lambda x: dview2.TimedeltaView(x, fmt='micros') for c in microsecond_cols}, subset=microsecond_cols)\
    .hide_columns(['is_origin', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru'])
# opp_bk.head(100).groupby(level=[0,1,2]).apply(lambda x: x.head(3))
# ov.opp_view

In [18]:
import math
opp_dur_buckets = {
    pd.Timedelta('0us'): CellView(),
    pd.Timedelta('50us'): CellView(bg=Colors.gold()),
    pd.Timedelta('100us'): CellView(bg=WebColors.sky_blue()),
    pd.Timedelta('500us'): CellView(bg=Colors.lightgreen_pastel())
}

edge_buckets = {
    -0.25: CellView(fw = 'bold', bg = Colors.darkred()),
    0.0: CellView(),
    0.25: CellView(fg=Colors.blue(), fw='bold'),
    0.50: CellView(fg=Colors.blue(), fw='bold', bg=WebColors.sky_blue()),
    1.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.gold()),
    2.0: CellView(fg=Colors.blue(), fw='bold', bg=Colors.lightgreen_pastel()),
}

opp_bk= opp_w_bk_root.copy()
# opp_bk['lst_delta_us'] = opp_bk['lst_delta'].apply(lambda x: x.microseconds)
opp_bk = opp_bk.loc[:,'+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',:]
# opp_w_bk['opp_start_eid'] = opp_w_bk.index.get_level_values('opp_start_eid')
# opp_bk = opp_bk.loc[opp_bk['opp_dur_lsn'] > pd.Timedelta(0, unit='us')]
ov = OppView(opp_bk.head(400), opp_durs=opp_dur_buckets, edges=edge_buckets, opp_origin_css='opp-origin')
# ov = OppView(opp_w_bk.head(120))
microsecond_cols = ['opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn']
ov.display(verbose=None,
           columns =['t_tod', 'bk_dur', 'opp_dur', 'opp_dur_fsn', 'opp_dur_lsn', 'opp_dur_thru', 'opp_dur_thru_lsn', 'edge', 'side', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1'])\
    .format({c: lambda x: '' if math.isnan(x.microseconds) else f'{dview.TimedeltaView(x).normal_microseconds():.0f}' for c in microsecond_cols}, subset=microsecond_cols)\
    .hide_columns('opp_dur')
# ov.opp_view

('border-top-style', 'solid')


In [2]:
market_dates = {pid: set(get_feather_dates(340, name = f'synthetic_book_{pid}', month=[5])) for pid in range(1, 2)}
market_dates = list(set.intersection(*[set(v) for k, v in market_dates.items()]))
symbols=[
    '+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',
    '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',
    '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1']

# opp = pd.read_feather('/home/john/spartan_store/pcaps/340/jupyter_cache/opp_apr_jun_jul.feather').set_index(['market_date', 'symbol', 'opp_start_eid'])
# opp_root = pd.read_feather('/home/john/spartan_store/pcaps/340/jupyter_cache/opp_root_apr_jun_jul.feather').set_index(['market_date', 'symbol', 'eid'])
opp_loader = SniperOppLoader(340, dates=market_dates, log_level=1)
opp_root = opp_loader.load_asset_opp_root('zs')
opp = opp_loader.load('zs')

[Info ] [2021-07-22 05:24:18.650001394]  |  Begin loading asset zs (None symbols, None dates)
[Info ] [2021-07-22 05:24:22.185623054]  |  End loading asset zs (None symbols, None dates).  Took 0 days 00:00:03.535093001


In [3]:
synth_bk = opp_loader.synth_bk()

[Info ] [2021-07-22 05:24:22.197057246]  |  Begin loading synth books (all on 2021-05-09, 2021-05-11, 2021-05-02, 2021-05-25, and 17 others)
[Info ] [2021-07-22 05:24:22.448704339]  |  [2021-05-11] Processing 2021-05-11 polygon 1[Info ] [2021-07-22 05:24:22.448836334]  |  [2021-05-02] Processing 2021-05-02 polygon 1[Info ] [2021-07-22 05:24:22.449003880]  |  [2021-05-25] Processing 2021-05-25 polygon 1[Info ] [2021-07-22 05:24:22.449590774]  |  [2021-05-19] Processing 2021-05-19 polygon 1[Info ] [2021-07-22 05:24:22.449421570]  |  [2021-05-23] Processing 2021-05-23 polygon 1[Info ] [2021-07-22 05:24:22.449225086]  |  [2021-05-04] Processing 2021-05-04 polygon 1[Info ] [2021-07-22 05:24:22.449952185]  |  [2021-05-30] Processing 2021-05-30 polygon 1[Info ] [2021-07-22 05:24:22.448689167]  |  [2021-05-09] Processing 2021-05-09 polygon 1
[Info ] [2021-07-22 05:24:22.450155572]  |  [2021-05-17] Processing 2021-05-17 polygon 1
[Info ] [2021-07-22 05:24:22.450373448]  |  [2021-05-06] Processi

In [8]:
opp_loader.synth_bk()[['id', 't_time', 'fs_time', 'ls_time', 'bk_dur', 'bk_dur_fsn', 'bk_dur_lsn', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1']]

id                        t_time                    fs_time                    ls_time                    bk_dur                  bk_dur_fsn                  bk_dur_lsn  bid_q_1  bid_p_1  ask_p_1  ask_q_1
market_date symbol                                             eid                                                                                                                                                                                                                   
2021-05-26  +(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 0          2 2021-05-26 21:45:19.713668391                        NaT                        NaT 0 days 00:03:36.366836966                         NaT                         NaT        1    -2.00     1.75        1
                                                               1113       2 2021-05-26 21:48:56.080505357 2021-05-26 21:48:56.081414 2021-05-26 21:48:56.081414 0 days 00:20:39.219951540   0 days 00:20:39.219042897   0 days 00:20:39.219042897        1    -2.50     1.25        1
                                                               2964       2 2021-05-26 22:09:35.300456897 2021-05-26 22:09:35.300713 2021-05-26 22:09:35.300713 0 days 00:00:00.018952220   0 days 00:00:00.018696117   0 days 00:00:00.018696117        1    -2.50      NaN        0
                                                               2974       2 2021-05-26 22:09:35.319409117 2021-05-26 22:09:35.319501 2021-05-26 22:09:35.319501 0 days 01:45:24.994605884   0 days 01:45:24.994514001   0 days 01:45:24.994514001        0      NaN      NaN        0
                                                               5531       2 2021-05-26 23:55:00.314015001 2021-05-26 23:55:00.314134 2021-05-26 23:55:00.314134 0 days 00:00:00.000004322 -1 days +23:59:59.999885323 -1 days +23:59:59.999885323        1    -2.75      NaN        0
...                                                                      ..                           ...                        ...                        ...                       ...                         ...                         ...      ...      ...      ...      ...
2021-05-31  +ZSQ1 -(ZSQ1-ZSU1) -ZSU1                           23180260   6 2021-06-01 18:20:16.068291999 2021-06-01 18:20:16.068608 2021-06-01 18:20:16.068608 0 days 00:09:44.002123504   0 days 00:09:44.001807503   0 days 00:09:44.001807503        1   -12.25     4.25        1
                                                               23183812   6 2021-06-01 18:30:00.070415503 2021-06-01 18:30:00.181173 2021-06-01 18:30:00.181957 0 days 01:00:00.233753228   0 days 01:00:00.122995731   0 days 01:00:00.122211731        1   -25.25    29.00        1
                                                               23184377   6 2021-06-01 19:30:00.304168731 2021-06-01 19:30:00.304291 2021-06-01 19:30:00.304291 0 days 00:00:03.668913052   0 days 00:00:03.668790783   0 days 00:00:03.668790783        1   -18.00    29.00        1
                                                               23184383   6 2021-06-01 19:30:03.973081783 2021-06-01 19:30:03.973204 2021-06-01 19:30:03.973204 0 days 00:36:51.561829522   0 days 00:36:51.561707305   0 days 00:36:51.561707305        1   -18.00    29.00        1
                                                               23184727   6 2021-06-01 20:06:55.534911305 2021-06-01 20:06:55.535045 2021-06-01 20:06:55.535045                       NaT                         NaT                         NaT        1   -18.00    29.00        1

[72783251 rows x 11 columns]

In [4]:
opp_summarizer.synth_bk().reset_index().to_feather('/home/john/spartan_store/pcaps/340/jupyter_cache/synth_bk_2021-05_01-31_p1-11.feather')
opp_root.reset_index().to_feather('/home/john/spartan_store/pcaps/340/jupyter_cache/opp_root_2021-05_01-31_p1-11.feather')
opp.reset_index().to_feather('/home/john/spartan_store/pcaps/340/jupyter_cache/opp_2021-05_01-31_p1-11.feather')

In [15]:
synth_bk.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 72783251 entries, (Timestamp('2021-05-09 00:00:00'), '+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)', 0) to (Timestamp('2021-05-03 00:00:00'), '+ZSQ1 -(ZSQ1-ZSU1) -ZSU1', 22779925)
Data columns (total 19 columns):
 #   Column      Dtype          
---  ------      -----          
 0   t_time      datetime64[ns] 
 1   id          int32          
 2   bid_p_1     float32        
 3   bid_q_1     uint32         
 4   bid_p_2     float32        
 5   bid_q_2     uint32         
 6   bid_p_3     float32        
 7   bid_q_3     uint32         
 8   ask_p_1     float32        
 9   ask_q_1     uint32         
 10  ask_p_2     float32        
 11  ask_q_2     uint32         
 12  ask_p_3     float32        
 13  ask_q_3     uint32         
 14  bk_dur      timedelta64[ns]
 15  fs_time     datetime64[ns] 
 16  ls_time     datetime64[ns] 
 17  bk_dur_lsn  timedelta64[ns]
 18  bk_dur_fsn  timedelta64[ns]
dtypes: datetime64[ns](3), float32(6)

In [16]:
import src.synthetic_book_merger5 as sbm5
synth_bk_date_groupby = opp_loader.synth_bk().groupby(level=[0])
# odg = opp.groupby(level=[0])
odg = opp.set_index(opp.index.set_names(['market_date', 'symbol', 'opp_start_eid'])).groupby(level=[0])
opp_w_bk_root = sbm5.SyntheticBookMerger(odg, synth_bk_date_groupby).merge(parallel=True, min_chunk_sz=5, parallel_min_sz=15)

[Diag ] [2021-07-22 07:31:57.403183657]  |  Begin merge opp books for 20 dates
[Diag ] [2021-07-22 07:31:57.403506606]  |  Processing in parallel: 4 chunks of size 5 (rem 0)
[Diag ] [2021-07-22 07:32:56.111678841]  |  Adding synth books for 2021-05-02 00:00:00
[Diag ] [2021-07-22 07:33:24.442671689]  |  Adding synth books for 2021-05-09 00:00:00
[Diag ] [2021-07-22 07:33:32.968239365]  |  Adding synth books for 2021-05-03 00:00:00
[Diag ] [2021-07-22 07:33:45.186546808]  |  Adding synth books for 2021-05-04 00:00:00
[Diag ] [2021-07-22 07:34:00.615010867]  |  Adding synth books for 2021-05-16 00:00:00
[Diag ] [2021-07-22 07:34:19.101653206]  |  Adding synth books for 2021-05-05 00:00:00
[Diag ] [2021-07-22 07:34:29.345217029]  |  Adding synth books for 2021-05-23 00:00:00
[Diag ] [2021-07-22 07:34:35.322911791]  |  Adding synth books for 2021-05-24 00:00:00
[Diag ] [2021-07-22 07:34:41.266785224]  |  Adding synth books for 2021-05-10 00:00:00
[Diag ] [2021-07-22 07:34:49.930440802]  | 

In [2]:
def get_row_selectors(df, win_cols, win_szs = [0,45,55,75,100]):
    win_cols = ensure_iterable(win_cols)
    rv= pd.DataFrame({c: np.digitize(df[c], win_szs) for c in win_cols}, index = df.index)
    for c in win_cols:
        rv[c] = rv[c].astype('category')
        rv[c] = rv[c].cat.rename_categories([f'win_{win_szs[i - 1]}' for i in rv[c].cat.categories])
#     print(rv.head(10))
    return rv
#     return pd.DataFrame({c: np.where(df[c] > 45, 'obtainable', 'unobtainable') for c in win_cols}, index = df.index)
#     print(type(obtainable))
#     return pd.DataFrame(data={win_col: obtainable})

opp_root_tmp = opp_root.loc['2021-05-12','+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)',:]\
    .drop(columns=[
        'cp1_pbook', 'cp1_pbook_dir', 'cp2_pbook', 'cp2_pbook_dir',
        'entry_ev', 'entry_pbook', 'entry_pbook_dir', 'entry_leg_books',
        'exit_event', 'exit_pbook', 'exit_pbook_dir', 'exit_leg_books', 'penalty_legs'])\
    [['t_tod', 'side', 'has_futures', 'min_days', 'max_days', 'min_fut_vol', 'entry_ticks', 'entry_pnl', 'entry_qty', 'is_direct', 'fsn_win', 'lsn_win']]
opp_root_tmp.sort_index(inplace=True)
opp_root_tmp['lsn_win_us'] = opp_root_tmp['lsn_win'].apply(lambda x: x.microseconds)
opp_root_tmp['fsn_win_us'] = opp_root_tmp['fsn_win'].apply(lambda x: x.microseconds)
win_szs = [0,45,100,200]
opp_root_tmp.drop(columns=['fsn_win', 'lsn_win'], inplace=True)
sdict =  dview.get_style_dict(opp_root_tmp, {x: '{:.2f}' for x in ['entry_pnl']})
sdict.update({'lsn_win_us': '{:,}'})
style = opp_root_tmp.style.format(sdict)
def get_win_props(idx):
    if idx == 0:
        return []
    return [('background', Colors.blue().blend(Colors.red(), win_szs[idx] / win_szs[-1]))]
style.set_table_styles([{'selector': f'.win_{win_szs[idx]}', 'props': get_win_props(idx)} for idx in range(0, len(win_szs))])
# style.set_table_styles([{'selector': f'.win_45', 'props': [('background', 'blue')]}])
# style.set_table_styles([{
#     'selector':'.obtainable',
#     'props': [('background', 'lightseagreen'), ('color', '#BBCC77'), ('font-weight', 'bold')]}])
style.set_td_classes(get_row_selectors(opp_root_tmp, ['fsn_win_us', 'lsn_win_us'], win_szs)).hide_index()
#     apply(get_lsn_win_selector, subset=['lsn_win', 'lsn_win_us'])\

t_tod,side,has_futures,min_days,max_days,min_fut_vol,entry_ticks,entry_pnl,entry_qty,is_direct,lsn_win_us,fsn_win_us
06:45.75,Buy,True,63,184,-1,1,12.50,1,True,"999,809",999809
11:04.46,Sell,True,63,184,-1,4,50.00,1,True,190,700
32:33.49,Sell,True,63,184,-1,1,12.50,1,True,"999,954",999996
39:40.11,Sell,True,63,184,-1,1,12.50,1,True,139,183
46:15.90,Sell,True,63,184,-1,1,12.50,1,True,41,86
01:20:26.07,Sell,True,63,184,-1,1,12.50,1,True,"998,919",37
01:20:52.99,Buy,True,63,184,-1,1,12.50,1,True,389,389
01:54:31.24,Sell,True,63,184,-1,1,12.50,1,True,37,56
02:22:50.77,Buy,True,63,184,-1,1,12.50,2,True,703,722
02:31:36.15,Buy,True,63,184,-1,1,12.50,1,True,28,49


In [15]:
delays = [pd.Timedelta(f'{x}us') for x in [0,100,200,300,400,500]]
# get_edge_per_delay(opp_w_bk, delays, .25).loc['2021-05-02':'2021-05-05','+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',:]
pnl_cols = []
for d in delays:
    pnl_cols.append(pnlview.fmt_pnl_col('nm', d))
    pnl_cols.append(pnlview.fmt_pnl_col('m', d))
shot_outcomes = pnlview.get_opp_pnl(opp_w_bk, .25, delays)[pnl_cols]\
    .groupby(['symbol', 'market_date'])\
    .sum()
# shot_outcomes = shot_outcomes.loc[symbols,:]
shot_outcomes.groupby(level=0).sum().head(11)

,0us_nm_pnl,0us_m_pnl,100us_nm_pnl,100us_m_pnl,200us_nm_pnl,200us_m_pnl,300us_nm_pnl,300us_m_pnl,400us_nm_pnl,400us_m_pnl,500us_nm_pnl,500us_m_pnl
symbol,,,,,,,,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),223.64,1144.76,223.64,1144.76,223.64,1144.76,236.14,1157.26,236.14,1157.26,148.64,1069.76
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSF2) -(ZSX1-ZSF2),224.90,544.10,224.90,544.10,224.90,544.10,224.90,544.10,224.90,544.10,212.40,531.60
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),426.94,789.46,426.94,789.46,426.94,789.46,426.94,789.46,426.94,789.46,439.44,801.96
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),516.90,722.10,516.90,722.10,516.90,722.10,516.90,722.10,516.90,722.10,529.40,734.60
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),218.86,437.74,218.86,437.74,218.86,437.74,218.86,437.74,218.86,437.74,181.36,400.24
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,422.04,625.04,422.04,625.04,422.04,625.04,422.04,625.04,422.04,625.04,422.04,625.04
+ZSN1 -(ZSN1-ZSQ1) -ZSQ1,286.80,324.56,286.80,324.56,286.80,324.56,286.80,324.56,286.80,324.56,286.80,324.56
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,655.40,935.40,692.90,972.90,680.40,960.40,680.40,960.40,680.40,960.40,692.90,972.90
+ZSN1 -(ZSN1-ZSX1) -ZSX1,300.60,352.52,300.60,352.52,300.60,352.52,300.60,352.52,300.60,352.52,300.60,352.52


In [14]:
hm = dview.Heatmap(steps={-10: Color('red'), 50: Color('black'), 400: Colors.lightgreen_pastel()}, interpolate=True)
pnl_cols = [pnlview.fmt_pnl_col('nm', d) for d in delays] + [pnlview.fmt_pnl_col('m', d) for d in delays]
pnl_col_styles = {pnlview.fmt_pnl_col('nm', d): '{:.2f}' for d in delays}
pnl_col_styles.update({pnlview.fmt_pnl_col('m', d): '{:.2f}' for d in delays})
shot_outcomes[pnl_cols].sort_values('500us_nm_pnl', ascending=False).style\
    .format(pnl_col_styles)\
    .apply(lambda x: x.apply(hm.to_lambda()))

,,0us_nm_pnl,100us_nm_pnl,200us_nm_pnl,300us_nm_pnl,400us_nm_pnl,500us_nm_pnl,0us_m_pnl,100us_m_pnl,200us_m_pnl,300us_m_pnl,400us_m_pnl,500us_m_pnl
symbol,market_date,,,,,,,,,,,,
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,2021-05-06 00:00:00,310.64,310.64,310.64,310.64,310.64,323.14,408.64,408.64,408.64,408.64,408.64,421.14
+(ZSN1-ZSX1) -(ZSN1-ZSF2) +(ZSX1-ZSF2),2021-05-06 00:00:00,285.78,285.78,285.78,285.78,285.78,298.28,361.02,361.02,361.02,361.02,361.02,373.52
+ZSN1 -(ZSN1-ZSU1) -(ZSU1-ZSX1) -ZSX1,2021-05-02 00:00:00,208.36,208.36,208.36,208.36,208.36,208.36,285.36,285.36,285.36,285.36,285.36,285.36
+ZSN1 -(ZSN1-ZSX1) -ZSX1,2021-05-06 00:00:00,204.60,204.60,204.60,204.60,204.60,204.60,209.32,209.32,209.32,209.32,209.32,209.32
+(ZSX1-ZSH2) +(ZSH2-ZSN2) -(ZSX1-ZSX2) +(ZSN2-ZSX2),2021-05-06 00:00:00,210.70,210.70,210.70,210.70,210.70,185.70,256.30,256.30,256.30,256.30,256.30,231.30
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,2021-05-02 00:00:00,156.08,156.08,156.08,156.08,156.08,156.08,212.08,212.08,212.08,212.08,212.08,212.08
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),2021-05-05 00:00:00,153.50,153.50,153.50,153.50,153.50,153.50,381.50,381.50,381.50,381.50,381.50,381.50
+ZSN1 -(ZSN1-ZSQ1) -(ZSQ1-ZSU1) -ZSU1,2021-05-06 00:00:00,141.30,141.30,141.30,141.30,141.30,141.30,176.30,176.30,176.30,176.30,176.30,176.30
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),2021-05-06 00:00:00,138.72,138.72,138.72,138.72,138.72,138.72,234.48,234.48,234.48,234.48,234.48,234.48


In [14]:
# shot_outcomes[[pnlview.fmt_pnl_col('nm', d) for d in delays]].groupby(level=[0]).sum()
# shot_outcomes[[pnlview.fmt_pnl_col('nm', d) for d in delays]].groupby(level=0).sum()

45

In [315]:
shot_outcomes[[pnlview.fmt_pnl_col('nm', d) for d in delays]].groupby(level=[0]).sum().head(2)
# shot_outcomes[[pnlview.fmt_pnl_col('nm', d) for d in delays]].groupby(level=0).sum()

,0us_nm_pnl,100us_nm_pnl,200us_nm_pnl,300us_nm_pnl,400us_nm_pnl,500us_nm_pnl
symbol,,,,,,
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1),891.36,891.36,903.86,903.86,916.36,741.36
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1),1850.16,1862.66,1862.66,1862.66,1862.66,1762.66


In [233]:
delays = [pd.Timedelta(f'{x}us') for x in [0,100,200,300,400,500]]
# delays = [pd.Timedelta(f'{x}us') for x in [0]]
# mkt_date_index = pd.Index(opp_w_bk.index.get_level_values(0))
known_syms = ['+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)']

# get_edge_per_delay(opp_w_bk, delays, .25).loc['2021-05-02':'2021-05-05','+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)',:]
pnl_cols = []
for d in delays:
    pnl_cols.append(fmt_pnl_col('nm', d))
    pnl_cols.append(fmt_pnl_col('m', d))
shot_outcomes = get_edge_per_delay(opp_w_bk, delays, .25)[pnl_cols]\
    .groupby(['symbol', 'market_date'])\
    .sum()

In [235]:
shot_outcomes
# shot_outcomes.format
#     .sort_values(['symbol', 'market_date'])[['symbol', 'market_date', '0us_nm_pnl', '0us_m_pnl']]

0us_nm_pnl  0us_m_pnl  100us_nm_pnl  100us_m_pnl  200us_nm_pnl  200us_m_pnl  300us_nm_pnl  300us_m_pnl  400us_nm_pnl  400us_m_pnl  500us_nm_pnl  500us_m_pnl
symbol                                             market_date                                                                                                                                                              
+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-Z... 2021-05-02        86.02     250.18         86.02       250.18         86.02       250.18         86.02       250.18         86.02       250.18         48.52       212.68
                                                   2021-05-03       -36.12     118.92        -36.12       118.92        -36.12       118.92        -36.12       118.92        -36.12       118.92        -48.62       106.42
                                                   2021-05-04       -30.08     225.28        -30.08       225.28        -30.08       225.28        -30.08       225.28        -30.08       225.28        -67.58       187.78
                                                   2021-05-05       153.50     381.50        153.50       381.50        153.50       381.50        153.50       381.50        153.50       381.50        153.50       381.50
                                                   2021-05-06        50.32     168.88         50.32       168.88         50.32       168.88         62.82       181.38         62.82       181.38         62.82       181.38
                                                   2021-05-09       258.48     550.32        258.48       550.32        258.48       550.32        258.48       550.32        258.48       550.32        233.48       525.32
                                                   2021-05-10        38.88     193.92         38.88       193.92         38.88       193.92         38.88       193.92         38.88       193.92         38.88       193.92
                                                   2021-05-11       149.16     550.44        149.16       550.44        161.66       562.94        161.66       562.94        161.66       562.94        149.16       550.44
                                                   2021-05-12       160.18     725.62        160.18       725.62        160.18       725.62        147.68       713.12        160.18       725.62        147.68       713.12
                                                   2021-05-13        61.02     225.18         61.02       225.18         61.02       225.18         61.02       225.18         61.02       225.18         23.52       187.68
+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)             2021-05-02        97.30     165.70         97.30       165.70         97.30       165.70         97.30       165.70         97.30       165.70        109.80       178.20
                                                   2021-05-03         2.94      23.46          2.94        23.46          2.94        23.46          2.94        23.46          2.94        23.46          2.94        23.46
                                                   2021-05-04        60.78     136.02         60.78       136.02         60.78       136.02         60.78       136.02         60.78       136.02         60.78       136.02
                                                   2021-05-05       127.20     229.80        127.20       229.80        127.20       229.80        127.20       229.80        127.20       229.80        127.20       229.80
                                                   2021-05-06       138.72     234.48        138.72       234.48        138.72       234.48        138.72       234.48        138.72       234.48        138.72       234.48
                                                   2021-05-09       326.46     511.14        326.46       511.14        326.46       511.14        326.46       511.14        326.46       511.14        288.96       473.64
                                                   2021-05-10       221.56     372.04 

In [ ]:

from bisect import bisect_left
# tmp_opps = opp_w_bk.copy()
def get_edge_per_delay(all_opps, delays, req_edge):
    def get_edge(opps, latency = pd.Timedelta('50us'), cpp = 50.0):
        def impl(opp):
            data = {}
            for delay in delays:
                shot_idx = bisect_left(opp['opp_dur_thru'], delay)
                if shot_idx == len(opp):
                    shot_idx -= 1
                shot_start_row = opp.iloc[shot_idx]
                if shot_start_row['opp_dur_thru'] < delay:
                    return None
                if shot_start_row['edge'] < req_edge:
                    return pd.DataFrame(data={
                        'raw_edge': 0.0,
                        'cash_edge': 0.0,
                        'cash_edge': 0.0,
                        'non_member_cash_edge': 0.0,
                        'member_cash_edge': 0.0,
                        'fill_eid': 0
                    }, index=[opp.index.get_level_values('opp_start_eid')[0]])

                fill_idx = bisect_left(opp.iloc[shot_idx:]['opp_dur_thru'], delay + latency)
                fill_idx += shot_idx
                if fill_idx == len(opp):
                    fill_idx -= 1
                fill_row = opp.iloc[fill_idx]
                if fill_row['opp_dur_thru'] < (delay + latency):
                    raise Exception('Unable to compute fill price')

                cash_edge = fill_row['edge'] * cpp,
                nm_out_col = f'{delay.microseconds}us_nm_pnl'
                m_out_col = f'{delay.microseconds}us_m_pnl'
                data[nm_out_col] = cash_edge - fill_row['non_member_fee']
                data[m_out_col] = cash_edge - fill_row['member_fee']
                data[f'{delay.microseconds}us_fill_eid'] = fill_row['synth_bk_eid']
            res = pd.DataFrame(data=data)
            return res

        res = opps.groupby(level=[0, 1, 2]).apply(impl).reset_index(level=[0,1,2], drop=True)
        res.index.rename('opp_start_eid', inplace=True)
        return res
    grp = all_opps.groupby(level=[0,1])
    return grp.apply(get_edge)

delays = [pd.Timedelta(f'{x}us') for x in [0,100,200,300,400,500]]
delays = [pd.Timedelta(f'{x}us') for x in [0]]
# mkt_date_index = pd.Index(opp_w_bk.index.get_level_values(0))
known_syms = ['+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)']

get_edge_per_delay(opp_w_bk, delays, 0.50)
# sym_date_shot_grps = shot_outcomes.loc['2021-05-02',known_syms,:].groupby(level=[0, 1])

# def get_sym_outcomes(sym_shots):
#     cols = {}
# #     if (mkt_date, sym) not in sym_date_shot_grps.groups():
# #         raise Exception('hey') 
# #     sym_shots = sym_date_shot_grps.get_group((mkt_date, sym))
#     def get_col_data(d, req_edge, col_name):
#         return (f'{col_name}_{d.microseconds}us_{req_edge:.2f}', f'{col_name}_pnl')
#     for d in delays:
#         for req_edge in [.25, .50]:
#             shot_outcomes = get_edge_per_delay(opp_w_bk, d, req_edge)
#             for mship in ['nm', 'm']:
#                 def get_date_edge(col_name):
#                     return shot_outcomes.groupby(level=[0])[col_name].sum()
#                 cols[f'{mship}_{d.microseconds}us_{req_edge:.2f}'] = get_date_edge(f'{mship}_pnl')
#     return cols
#     # sym_summ.groupby(level=[0]).sum().style.format(sdict).set_caption(f'{sym}: {sym_summ["non_member_cash_edge"].sum():.2f}')

# # cols = []
# # for d in delays:
# #     for req_edge in [.25, .50]:
# #         cols.append(f'nm_{d.microseconds}us_{req_edge:.2f}')
# #         cols.append(f'm_{d.microseconds}us_{req_edge:.2f}')
# pd.DataFrame(data=get_sym_outcomes(sym_date_shot_grps.get_group(('2021-05-02', known_syms[0]))))

# #         show_symbol_summ('+(ZSN1-ZSQ1) +(ZSQ1-ZSU1) -(ZSN1-ZSX1) +(ZSU1-ZSX1)')
# #         show_symbol_summ('+(ZSN1-ZSQ1) -(ZSN1-ZSX1) +(ZSQ1-ZSX1)')

# # sdict = {'non_member_cash_edge': '{:.2f}', 'member_cash_edge': '{:.2f}'}
# # display(shot_outcomes[['non_member_cash_edge', 'member_cash_edge']].groupby(level=[0]).sum().style.format(sdict).set_caption('cummulative'))
# # shot_outcomes.loc['2021-05-02',:,:]

# # for delay in [pd.Timedelta(f'{x}us') for x in [0,100,200,300,400,500]]:
# # #     shot_outcomes = pd.DataFrame(data={'edge': get_opp_shot_outcomes(opp_w_bk, delay)})
# # #     print(len(shot_outcomes.sum()))
# #     shot_outcomes = get_opp_shot_outcomes(opp_w_bk, delay)
 #     print(shot_outcomes.sum())